CVD
-------------

<p>Retours à faire sur le présent travail<br>

L'importation a été **mal faite** et de <strong> fausses données </strong> se sont incrustées. Soit trouver une manière de nettoyer après coup ces données (substancielles:4%), soit trouver un autre moyen d'effectuer l'importation.</p>

En quoi consistes les factures avec plusieures dates?


Importation des données
===========

In [6]:
import pandas as pd
import numpy as np

In [25]:
path='Documents/travail/StartUp/CVD/cmptants/20052021.1.xls'
#pd.read_excel(path,sheet_name='20052021Clean.csv')#error in directory
import os
os.getcwd()#Sortie: '/Users/charlottebacave/Documents/travail/prog'

'/Users/charlottebacave/Documents/travail/prOg'

In [26]:
dataX='/Users/charlottebacave'+'/'+path
df=pd.read_excel(dataX,sheet_name='20052021Clean.csv')
#joli

In [27]:
#Corrigeons le titre des attributs
Zchap=df.columns[1][-2]#é est devenu un Z étrange
str0=list(df.columns)
def fNpReplace(x): 
    x=np.char.replace(x,old=Zchap,new='e')#was at origin an é; we want to avoid further dificulties
    return x
#print(x)
names=np.apply_along_axis(fNpReplace,arr=str0,axis=0)
df.columns=names

In [23]:
#Renomons finalement(l'étape précédente était pour le principe) les columns pour être facilement appelées
x='Dencaiss '+'Dentree '+' '.join(np.char.strip(names[2:7]))+' '+'Mapplique'
#(np.char.strip:We noticed there was a disturbing space at the end of one word) 
names=x.split(' ')
names
df.columns=names
df.head()

,Dencaiss,Dentree,Montant,Arrond.,Client,Quart,Facture,Mapplique
0,2005-06-01 00:00:00,2005-06-01 00:00:00,8,NaN,Carol Wright (291/R602),827.0,16549.0,8
1,2005-06-01 00:00:00,2005-06-01 00:00:00,0,NaN,Clinique vŽtŽrinaire Dorval (1/R9999),827.0,16550.0,0
2,2005-06-01 00:00:00,2005-06-01 00:00:00,32.49,NaN,Helmut Thievessen (1219/R0),827.0,16570.0,32.49
3,2005-06-01 00:00:00,2005-06-01 00:00:00,0,NaN,Karin Bonert (136/R1279),827.0,16551.0,0
4,2005-06-01 00:00:00,2005-06-01 00:00:00,17.96,NaN,Sisto Maiello (589/R176),827.0,16571.0,17.96


In [734]:
t=list(df.columns)
t.pop(3)
t.insert(3,'Arrond')
df.columns=t
#J'eus prefere un t.remove('Arrond.'), mais la methode renvois une erreur lorsque le terme a enlever est absent.

## erreurs dans l'importation/ texte insensé ou superflu
Nous elaborons une liste au fur et a mesure des premieres observations puis apportons les corrections necessaires<br>
- Zchap:Z avec accent èé
- Ê: '1Ê' et '3Ê'
- Date: heure 00:00:00 a la fin, 


## Ajustement des dates

In [ ]:
from dateutil.parser import parse

In [ ]:
#code de https://codereview.stackexchange.com/questions/172921/parse-through-text-and-extract-dates-in-uniform-format
#de jss367, sur stackexchange


import re

test_cases = ['04/30/2009', '06/20/95', '8/2/69', '1/25/2011', '9/3/2002', '4-13-82', 'Mar-02-2009', 'Jan 20, 1974',
        'March 20, 1990', 'Dec. 21, 2001', 'May 25 2009', '01 Mar 2002', '2 April 2003', '20 Aug. 2004',
        '20 November, 1993', 'Aug 10th, 1994', 'Sept 1st, 2005', 'Feb. 22nd, 1988', 'Sept 2002', 'Sep 2002',
        'December, 1998', 'Oct. 2000', '6/2008', '12/2001', '1998', '2002']

# Create a dictionary to convert from month names to numbers (e.g. Jan = 01)
month_dict = dict(jan='01', feb='02', mar='03', apr='04', may='05', jun='06', jul='07', aug='08', sep='09',
                  oct='10', nov='11', dec='12')


def word_to_num(string):
    """
    This function converts a string to lowercase and only accepts the first three letter.
    This is to prepare a string for month_dict
    Example:
        word_to_num('January') -> jan
    """

    s = string.lower()[:3]
    return month_dict[s]


def date_converter(line):
    """
    This function extracts dates in every format from text and converts them to YYYYMMDD.
    Example:
        date_converter("It was the May 1st, 2009") -> 20090501
    """
    results = []
    day = '01'
    month = '01'
    year = '1900'
    # If format is MM/DD/YYYY or M/D/YY or some combination
    regex = re.search('([0]?\d|[1][0-2])[/-]([0-3]?\d)[/-]([1-2]\d{3}|\d{2})', line)
    # If format is DD Month YYYY or D Mon YY or some combination, also matches if no day given
    month_regex = re.search(
        '([0-3]?\d)\s*(Jan(?:uary)?(?:aury)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug('
        '?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?(?:emeber)?).?,?\s([1-2]\d{3})',
        line)
    # If format is Month/DD/YYYY or Mon/D/YY or or Month DDth, YYYY or some combination
    rev_month_regex = re.search(
        '(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct('
        '?:ober)?|Nov(?:ember)?|Dec(?:ember)?).?[-\s]([0-3]?\d)(?:st|nd|rd|th)?[-,\s]\s*([1-2]\d{3})',
        line)
    # If format is any combination of just Month or Mon and YY or YYYY
    no_day_regex = re.search(
        '(Jan(?:uary)?(?:aury)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sept?('
        '?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?(?:emeber)?).?,?[\s]([1-2]\d{3}|\d{2})',
        line)
    # If format is MM/YYYY or M YYYY or some combination
    no_day_digits_regex = re.search('([0]?\d|[1][0-2])[/\s]([1-2]\d{3})', line)
    # If format only contains a year. If year is written alone it must be in form YYYY
    year_only_regex = re.search('([1-2]\d{3})', line)
    if regex:
        day = regex.group(2)
        month = regex.group(1)
        year = regex.group(3)
    elif month_regex:
        day = month_regex.group(1)
        month = word_to_num(month_regex.group(2))
        year = month_regex.group(3)
    elif rev_month_regex:
        day = rev_month_regex.group(2)
        month = word_to_num(rev_month_regex.group(1))
        year = rev_month_regex.group(3)
    elif no_day_regex:
        month = word_to_num(no_day_regex.group(1))
        year = no_day_regex.group(2)
    elif no_day_digits_regex:
        month = no_day_digits_regex.group(1)
        year = no_day_digits_regex.group(2)
    elif year_only_regex:
        year = year_only_regex.group(0)
    # Make sure all variables have correct number, add zeros if necessary
    month = month.zfill(2)
    day = day.zfill(2)
    if day == '00':
        day = '01'
    if year is not None and len(year) == 2:
        year = '19' + year
    results.append(year + month + day)
    return results


test_run = [date_converter(w) for w in test_cases]
print(test_run)

In [774]:
a=pd.DatetimeIndex(df.Dencaiss[0:100])
# np.range(len(a))[a.isna()]


AttributeError: module 'numpy' has no attribute 'range'

In [790]:
range(0,len(a))[a.isna()]

TypeError: only integer scalar arrays can be converted to a scalar index

'29-dŽc.-2005'

In [811]:

test_cases = ['04/30/2009', '06/20/95', '8/2/69', '1/25/2011', '9/3/2002', '4-13-82', 'Mar-02-2009', 'Jan 20, 1974',
              'March 20, 1990', 'Dec. 21, 2001', 'May 25 2009', '01 Mar 2002', '2 April 2003', '20 Aug. 2004',
              '20 November, 1993', 'Aug 10th, 1994', 'Sept 1st, 2005', 'Feb. 22nd, 1988', 'Sept 2002', 'Sep 2002',
              'December, 1998', 'Oct. 2000', '6/2008', '12/2001', '1998', '2002']
test_cases=df.Dencaiss[0:10]
for date_string in test_cases:
    print(date_string, parse(date_string))#.strftime("%Y%m%d"),)

TypeError: Parser must be a string or character stream, not datetime

########################################Observations instinctives sur les données/n

#

#

#

#

#######################




## NAN

In [ ]:
df.isna().sum(axis=0)
#Dencaiss=Dentree=Montant=Client=Quart=890 Nan: On se doute qu'il y a un lien dans l'avenance de NaN dans ces attributes
#Arrond=18966, Il y a beaucoup d'absences d'arrondis (arrondis=0)
#Facture=Mapplique=42 ??

In [738]:
# #Nan:Facture = #Nan:Mapplique = 42 ??
df[df.Facture.isna()]
#Facture.isna() == Mapplique.isna()
#Dans les cas ou Facture et/ou Mapplique sont NaN, on remarque que le client est souvent un employé de l'entreprise.
#Aussi, à un montant positif correspond le même montant negatif a la meme date: probablement des transactions particulieres, annulees, etc
#Cependant, ce n'est pas toujours le cas.

#Une analyse des nombres negatifs dans 'Montant' menera peut 
#Donc:
#dfT=df.loc[df.Facture.dropna().index]

,Dencaiss,Dentree,Montant,Arrond,Client,Quart,Facture,Mapplique
846,25-oct.-2005,25-oct.-2005,2.4,NaN,Lote Pasat (1777/Raa),925.0,NaN,NaN
2912,18-sept.-2006,18-sept.-2006,120,NaN,Sandrine Vigneron (3125/R0),1151.0,NaN,NaN
2913,18-sept.-2006,18-sept.-2006,-120,NaN,Sandrine Vigneron (3125/R0),1151.0,NaN,NaN
2940,22-sept.-2006,22-sept.-2006,0.1,NaN,Animatch (2652/R0),1156.0,NaN,NaN
2941,22-sept.-2006,22-sept.-2006,-0.1,NaN,Animatch (2652/R0),1156.0,NaN,NaN


In [859]:
#Mappliquee vs Montant
df[df.Mapplique.isna()].Montant.isna().sum()# Mapplique.isna() => (Montant!=Nan)
df[df.Montant.isna()].Mapplique.isna().sum()# Montant.isna() => (Mapplique!=Nan)
dfg=pd.DataFrame([df.Mapplique,df.Montant]).transpose()
dfg.dropna().sum(axis=1).eq(0).sum()-dfg.dropna().shape[0]
#Tous les Mapplique et Montant sont égaux sauf NaN presents dans l'un d'eux, alors ils sont inegaux
#En francais: sauf Nan, la caisse balance.
#Avec Mapplique.isna() == Facture.isna()
#Tout Montant avec facture egal son Mapplique: F=>Mapplique, Mapplique+Montant=> Mapplique==Montant
#Donc on peut retirer du nettoyage de ces attributs toutes les colonnes !(Montant.isna() ou Facture.isna())

0

In [860]:
dr=(df.Montant.isna() | df.Facture.isna()).eq(False)#dr:data a retirer
n=dr.sum()
df.shape[0]-n#il en reste 932
dfC=df[dr.eq(False)]#df corrige

### Imp0rtant:
1k donnees jetées sur 25k, ce n'est pas si mal pour une première expérience.

Le but du travail est de développer une série chronologique sur les données. Regardons ce qu'on peut faire avec ces données fortement tronquées, avant de procéder à davantage de nettoyage. Cela ne nuira pas trop à l'exercice non plus car il reste à nettoyer les Dates, travail de string similaire à celui à faire sur les données jetées pour les réintroduire.

## Montants NULS

In [725]:
#Aussi, des montants de 0$??
(df[df.Montant==0]).shape#Il y en a quand meme 378
Zero=df[df.Montant==0]#on remarque que plusieurs correspondent a des Mapplique !=0
iZ=Zero[Zero.Mapplique!=0].index#149,203,616,1356,1382,...
df.loc[146:151]
df.loc[201:205]
df.loc[613:620]# On voit que a des 0 dans Montant on a des Mapplique annules par la suite.
#Attardons nous aux cas de nombres negatifs, cela reglera du meme coup les 0 dans Montant

,Dencaiss,Dentree,Montant,Arrond,Client,Quart,Facture,Mapplique
613,13-sept.-2005,13-sept.-2005,25.18,NaN,Dorothy Cameron (376/Rgg),896.0,18547.0,25.18
614,13-sept.-2005,13-sept.-2005,0,NaN,HŽlne Gauvin (1184/R1698),896.0,18537.0,0
615,13-sept.-2005,13-sept.-2005,21.5,NaN,Jessica Barthe (2277/Rbb),896.0,18531.0,21.5
616,13-sept.-2005,13-sept.-2005,0,NaN,Marie-HŽlne Thomas (1728/R1717),896.0,16934.0,16.78
617,NaN,NaN,NaN,NaN,NaN,NaN,18546.0,-16.78
618,13-sept.-2005,13-sept.-2005,21.87,NaN,Sylvie Breton (1335/R1328),896.0,18527.0,21.87
619,14-sept.-2005,14-sept.-2005,15.99,NaN,Carol Wright (291/R602),897.0,18551.0,15.99
620,14-sept.-2005,14-sept.-2005,26.08,NaN,Christine Matte (190/R0),897.0,18549.0,26.08


## Montants NEGATIFS

In [740]:
# montants sont négatifs
numdf=pd.to_numeric(df.Montant.dropna(),errors='coerce')
unI=numdf[numdf.isna()].index
df.loc[unI]
#Hmmmm, on voit que Montant contient des abberations: '3Ê' ou '1Ê' suivit d'un nombre sense. 
#QUESTION: erreur d'importation ou dans les données d'origine

,Dencaiss,Dentree,Montant,Arrond,Client,Quart,Facture,Mapplique
1246,06-janv.-2006,06-janv.-2006,"3Ê184,75 $",NaN,Linda Richer (188/R1500),975.0,15533.0,40.52
3619,05-janv.-2007,05-janv.-2007,"1Ê349,70 $",NaN,Linda Richer (188/R1500),1226.0,20651.0,62.26
4392,2007-05-15 00:00:00,2007-05-15 00:00:00,"1Ê500,00 $",NaN,Jo Anne Prima (103/R25),1317.0,29262.0,166.35
4543,2007-06-06 00:00:00,2007-06-06 00:00:00,"3Ê840,09 $",NaN,Royal Canin (3466/Rcie),1332.0,30094.0,199.41
4563,2007-06-06 00:00:00,2007-06-06 00:00:00,"-3Ê840,09 $",NaN,Royal Canin (3466/Rcie),1332.0,30094.0,-199.41
6377,01-avr.-2008,01-avr.-2008,"1Ê058,23 $",NaN,Barbara Chavez (3886/Rjj),1540.0,37329.0,"1Ê058,23 $"
9374,11-sept.-2009,11-sept.-2009,"1Ê000,00 $",NaN,Jo Anne Prima (103/R25),1906.0,41016.0,51.94
11725,01-nov.-2010,01-nov.-2010,"1Ê000,00 $",NaN,Michael Bisanti (979/R2066),2185.0,49522.0,8
16154,2013-06-14 00:00:00,2013-06-14 00:00:00,"1Ê077,15 $",-0.01,Reg Wells (881/R1655),2839.0,78879.0,296.99
21461,08-fŽvr.-2017,08-fŽvr.-2017,"1Ê330,05 $",0.02,Patrick Burns (4682/R738),3761.0,105895.0,170.51


# non CLASSE

In [521]:
#Les redondances dans les numeros de facture proviennent de payements par bouts
#Les factures peuvent-elles servire comme index?

len(df.Facture.unique())/len(df.Facture)#96%
#Nous pensions à tort que les factures étaient uniques; tachons de trouver les redondances (4%)
s=df.Facture.value_counts()
s.gt(1).sum()#670
s.gt(2).sum()#500 environ=2
s.gt(3).sum()#140 ==3
s.eq(4).sum()#21
s.eq(5).sum()#8
s.gt(5).sum()#8
#Concentrons nous alors sur les 8 plus grandes factures
m=df.Facture.eq(s.index[0])
df[m].shape[0]#pas d'erreur
df[m]#On remarque que c'etait toujours la meme cliente Noelle Hannibal, la meme annee, Mapplique sont des montants ronds
df[df.Facture.eq(s.index[1])]#Maureen Cannon, pas les memes dates, Mapplique=(300,50,100,...)
x=2

In [522]:
#Mapplique NaN
dfnw=df.Mapplique.dropna()
dfnw=pd.to_numeric(dfnw, errors='coerce')
l1=len(dfnw)
l1-len(dfnw.dropna())#10 erreurs dans dfnw:c negligeable

10

In [523]:
#Arrondis
df.Arrond.isna().mean()#75%
df.Arrond.unique()
df.Arrond.value_counts()#.01:1750, .02:1658, -.02:1502, -.01:1486
df.Arrond.dropna().mean()#.09%
#Nous ne pensons pas qu'une augmentation de valeur en moyenne de .09% dans le cas d'arondis soit significatif.
#D'experience, un test non-paramétrique (H0:somme=0, H1:somme>0) ne sera pas positif et je ne vois pas d'hypotese a faire sur la distribution 

0.0009005628517823545

24430

In [525]:
dfg.isna().sum(axis=0)
#On a 20x plus de nan dans Montant que dans Mapplique

Mapplique     42
Montant      890
dtype: int64

In [606]:
i=df.Montant[df.Montant.isna()].index
test=df.loc[i]
test.isna().apply(np.mean,axis=0)
#On voit que tous les attribus sont pleins de NaN sauf Facture et Mapplique
test.index[0:10]# Int64Index([54, 61, 107, 108, 110, 111, 118, 119, 150, 204], dtype='int64')
df.loc[52:63]#On voit que NaN 54 et 61 correspondent a des factures incompletes
#Une facon de corriger serait d'ajouter les Mapplique de 'test' au Mapllique de l'index precedant
vAdd=df.Mapplique[test.index].values+df.Mapplique[test.index-1].values


TypeError: can only concatenate str (not "float") to str

In [687]:
#Nous reperons certaines valeurs non-numeriques: Factures 85902,86157 et 10628: elles  

a=df.Mapplique[test.index].values
b=df.Mapplique[test.index-1].values
iAnormal=np.isnan(pd.to_numeric(a,errors='coerce'))
iAnormal=df.Mapplique[test.index][iAnormal].index
df.loc[iDf[0]-5:iDf[0]+5]
df.loc[17495:17497].Facture.values
#les factures 85901,85902 et 86157 ont comme client l'entreprise 
#elles sont balencees (presence d'un -x==y+z)
#On en deduit que le '2Ê'present dans Factures:85902 et 86157 sont des erreurs, a eliminer
df.loc[iDf[-1]-3:iDf[-1]+3]
df[df.Mapplique==df.Mapplique[21461]-159.52]#Nous ne trouvons par le sens de la donnee
df.loc[iDf[-1]]#21462:Mapplique=1Ê159,52$, Facture=10628

Dencaiss            NaN
Dentree             NaN
Montant             NaN
Arrond              NaN
Client              NaN
Quart               NaN
Facture          106028
Mapplique    1Ê159,52 $
Name: 21462, dtype: object

In [690]:
df.loc[iDf[0]-5:iDf[0]+5]



,Dencaiss,Dentree,Montant,Arrond,Client,Quart,Facture,Mapplique
17490,2014-06-17 00:00:00,2014-06-17 00:00:00,17.2,NaN,Jenny Corbeil (80/R503),3092.0,86099.0,17.2
17491,2014-06-17 00:00:00,2014-06-17 00:00:00,96.1,NaN,Melanie Chartrand (187/R142),3092.0,86107.0,96.1
17492,2014-06-17 00:00:00,2014-06-17 00:00:00,20,-0.01,Vente comptoir,3092.0,86092.0,20.01
17493,2014-06-18 00:00:00,2014-06-18 00:00:00,7.7,0.01,Arlene Morris (814/R2226),3093.0,86139.0,7.69
17494,2014-06-18 00:00:00,2014-06-18 00:00:00,95.2,-0.02,Bonnie Spencer (7138/Rjj),3093.0,86144.0,95.22
17495,2014-06-18 00:00:00,2014-06-18 00:00:00,0,NaN,Centre VŽtŽrinaire DMV (1492/Rvet),3093.0,85901.0,92.97
17496,NaN,NaN,NaN,NaN,NaN,NaN,85902.0,"2Ê153,40 $"
17497,NaN,NaN,NaN,NaN,NaN,NaN,86157.0,"-2Ê246,37 $"
17498,2014-06-18 00:00:00,2014-06-18 00:00:00,17.2,NaN,Franoise Proulx (6902/R8),3093.0,86155.0,17.2
17499,2014-06-18 00:00:00,2014-06-18 00:00:00,40.25,-0.02,Gabrielle Patulli (4603/Rkk),3093.0,86129.0,40.27


#######################

# Analyse

####################